### -Open with Salesforce Environment-

In [385]:
import pandas as pd
from datetime import datetime, timezone
from dateutil.relativedelta import relativedelta
from collections import OrderedDict
from simple_salesforce import Salesforce, SalesforceLogin, SFType

In [386]:
username = 'adriel.piacquadio@engbim.com'
password = 'Adriel2021'
security_token = 'UiFCCsvyGhC115hMDBpeeC6K'

sf = Salesforce(username=username, password=password, security_token=security_token)
print(sf)

### Query for Opportunities

In [387]:
def query_salesforce_opps(sf, date):
    # Realizar la consulta SOQL con la fecha proporcionada
    query = f"SELECT Project__c, Id, Name, CreatedDate, Proposal_Delivery_Date__c, Internal_Owner__c, Amount, Deal_Type__c, CloseDate, Estimator__c, IsWon, Division__c, Trades__c, Project_stage__c, End_Market__c, Point_of_Contact_within_Account__c, Customer_Classification__c, StageName FROM Opportunity WHERE Project_stage__c != 'Operations' AND  CreatedDate > {date}"
    response = sf.query(query)
    
    # Extraer registros y crear DataFrame
    records = response['records']
    df = pd.DataFrame(records)
    
    # Eliminar la columna 'attributes'
    df.drop(columns=['attributes'], inplace=True)
    
    return df

In [388]:
def query_salesforce_opps2(sf, date1, date2):
    # Realizar la consulta SOQL con la fecha proporcionada
    query = f"SELECT Project__c, Id, Name, CreatedDate, Proposal_Delivery_Date__c, Internal_Owner__c, Amount, Deal_Type__c, CloseDate, Estimator__c, IsWon, Division__c, Trades__c, Project_stage__c, End_Market__c, Point_of_Contact_within_Account__c, Customer_Classification__c, StageName FROM Opportunity WHERE Project_stage__c != 'Operations' AND  CreatedDate > {date1} AND  CreatedDate < {date2}"
    response = sf.query(query)
    
    # Extraer registros y crear DataFrame
    records = response['records']
    df = pd.DataFrame(records)
    
    # Eliminar la columna 'attributes'
    df.drop(columns=['attributes'], inplace=True)
    
    return df

df_opps_00 = query_salesforce_opps2(sf, '2021-09-08T00:41:46.000+0000', '2022-01-17T12:47:33.000+0000')

In [389]:
# Fecha inicial para la consulta
date_opps = '2021-01-01T00:00:00.000+0000'
all_df_opps = []

while True:
    # Obtener los datos desde Salesforce
    df_opps = query_salesforce_opps(sf, date_opps)
    
    # Si no hay más datos, salir del bucle
    if df_opps.empty:
        break
    
    # Agregar el DataFrame actual a la lista de todos los DataFrames
    all_df_opps.append(df_opps)
    
    # Actualizar la fecha para la siguiente consulta
    date_opps = df_opps['CreatedDate'].max()
    
    # Verificar si se ha alcanzado el límite de 2000 registros
    if len(df_opps) < 2000:
        break

In [390]:
# Combinar todos los DataFrames en uno solo
df_opps_combined = pd.concat(all_df_opps, ignore_index=True)

# Combinar df_opps_00 con df_opps_combined
df_opps_final = pd.concat([df_opps_00, df_opps_combined], ignore_index=True)
df_opps_final = df_opps_final.drop_duplicates()

In [391]:
#df_opps.to_csv(r'C:\Users\AdrielPiacquadioENG\Desktop\Estimation\Predicciones\Forecasting Opps\Opportunities.csv', index=False)

### Query for Projects

In [392]:
def query_salesforce_proj(sf, id):
    # Realizar la consulta SOQL con la fecha proporcionada
    query = f"SELECT Id, Name, Project_State__c FROM Project__c WHERE Id >= '{id}'"
    response = sf.query(query)
    
    # Extraer registros y crear DataFrame
    records = response['records']
    df = pd.DataFrame(records)
    
    # Eliminar la columna 'attributes'
    df.drop(columns=['attributes'], inplace=True)
    
    return df

In [393]:
# Id inicial para la consulta
id_proj = 'a0p7V000007JxehQAC'
all_df_proj = []

while True:
    # Obtener los datos desde Salesforce
    df_proj = query_salesforce_proj(sf, id_proj)
    
    # Si no hay más datos, salir del bucle
    if df_proj.empty:
        break
    
    # Agregar el DataFrame actual a la lista de todos los DataFrames
    all_df_proj.append(df_proj)
    
    # Actualizar el Id para la siguiente consulta
    id_proj = df_proj['Id'].max()
    
    # Verificar si se ha alcanzado el límite de 2000 registros
    if len(df_proj) < 2000:
        break

In [394]:
df_proj_final = pd.concat(all_df_proj, ignore_index=True)
df_proj_final = df_proj_final.drop_duplicates()

In [395]:
#df_proj.to_csv(r'C:\Users\AdrielPiacquadioENG\Desktop\Estimation\Predicciones\Forecasting Opps\Projects.csv', index=False)

### Query for Clients & Contacts

In [396]:
def query_salesforce_acc(sf, id):
    # Realizar la consulta SOQL con la fecha proporcionada
    query = f"SELECT AccountId, Title, Name, Id, Account.Industry, Account.Name, Account.Sales_Division__c, Account.Type, Account.Type__c, Account.BillingAddress FROM Contact WHERE Id >= '{id}' ORDER BY Id ASC"
    response = sf.query(query)
    
    # Extraer registros y crear DataFrame
    records = response['records']
    df = pd.DataFrame(records)
    
    # Eliminar la columna 'attributes'
    df.drop(columns=['attributes'], inplace=True)

    # Extraer datos anidados en la columna 'Account' y convertirlos en columnas separadas
    account_data = df['Account'].apply(pd.Series)  # Esto convierte diccionarios en columnas
    account_data.columns = ['Account.' + col for col in account_data.columns]  # Prefijar con 'Account.'
    account_data.drop(columns=['Account.attributes'], inplace=True)
    
    # Concatenar los datos de Account al DataFrame principal
    df = pd.concat([df.drop(columns=['Account']), account_data], axis=1)
    
    return df

In [397]:
# Id inicial para la consulta
id_acc = '0030e00002GntyVAAR'
all_df_acc = []

while True:
    # Obtener los datos desde Salesforce
    df_acc = query_salesforce_acc(sf, id_acc)
    
    # Si no hay más datos, salir del bucle
    if df_acc.empty:
        break
    
    # Agregar el DataFrame actual a la lista de todos los DataFrames
    all_df_acc.append(df_acc)
    
    # Actualizar el Id para la siguiente consulta
    id_acc = df_acc['Id'].max()
    
    # Verificar si se ha alcanzado el límite de 2000 registros
    if len(df_acc) < 2000:
        break

df_acc_combined = pd.concat(all_df_acc, ignore_index=True)

In [398]:
# Get not accesible IDs

query_acc_00 = f"SELECT AccountId, Title, Name, Id, Account.Industry, Account.Name, Account.Sales_Division__c, Account.Type, Account.Type__c, Account.BillingAddress FROM Contact WHERE Id >= '0030P00001t16W8QAI' AND Id <= '0030P00002HHH52QAH' ORDER BY Id ASC"
response_acc_00 = sf.query(query_acc_00)

# Extraer registros y crear DataFrame
records_acc_00 = response_acc_00['records']
df_acc_00 = pd.DataFrame(records_acc_00)

# Eliminar la columna 'attributes'
df_acc_00.drop(columns=['attributes'], inplace=True)

# Extraer datos anidados en la columna 'Account' y convertirlos en columnas separadas
account_data_00 = df_acc_00['Account'].apply(pd.Series)  # Esto convierte diccionarios en columnas
account_data_00.columns = ['Account.' + col for col in account_data_00.columns]  # Prefijar con 'Account.'
account_data_00.drop(columns=['Account.attributes'], inplace=True)

# Concatenar los datos de Account al DataFrame principal
df_acc_00 = pd.concat([df_acc_00.drop(columns=['Account']), account_data_00], axis=1)

In [399]:
# Combinar df_acc_00 con df_acc_combined
df_acc_final = pd.concat([df_acc_00, df_acc_combined], ignore_index=True)

# Función para obtener el valor de 'state' de un OrderedDict
def get_state_value(x):
    if isinstance(x, OrderedDict):
        return x.get('state', None)
    else:
        return None

# Aplicar la función a la columna 'Account.BillingAddress'
df_acc_final['Account.BillingAddress'] = df_acc_final['Account.BillingAddress'].apply(get_state_value)

df_acc_final = df_acc_final.drop_duplicates()

In [400]:
#df_acc.to_csv(r'C:\Users\AdrielPiacquadioENG\Desktop\Estimation\Predicciones\Forecasting Opps\ContactsAccounts.csv', index=False)

### Joining SOQL Objects: Project - Opp. - Account - Contact

In [401]:
#df_opps = pd.read_csv(r'C:\Users\AdrielPiacquadioENG\Desktop\Estimation\Predicciones\Forecasting Opps\Opportunities.csv')
#df_acc = pd.read_csv(r'C:\Users\AdrielPiacquadioENG\Desktop\Estimation\Predicciones\Forecasting Opps\ContactsAccounts.csv')
#df_proj = pd.read_csv(r'C:\Users\AdrielPiacquadioENG\Desktop\Estimation\Predicciones\Forecasting Opps\Projects.csv')

In [402]:
df_opps_final = df_opps_final.rename(columns = {'Project__c':'ProjectId', 'Id':'OppId', 'Name':'Opportunity', 'Proposal_Delivery_Date__c':'DeliveryDate',
                                    'Internal_Owner__c':'InternalOwner', 'Sales_Division__c':'SalesDivision', 
                                    'Deal_Type__c':'DealType', 'Estimator__c':'Estimator', 'Division__c':'Division',
                                    'Trades__c':'Trades', 'Project_stage__c':'ProjectStage', 'End_Market__c':'EndMarket',
                                    'Point_of_Contact_within_Account__c':'ContactId', 'Customer_Classification__c':'CustomerClass'})

In [403]:
print(df_opps_final.columns)

Index(['ProjectId', 'OppId', 'Opportunity', 'CreatedDate', 'DeliveryDate',
       'InternalOwner', 'Amount', 'DealType', 'CloseDate', 'Estimator',
       'IsWon', 'Division', 'Trades', 'ProjectStage', 'EndMarket', 'ContactId',
       'CustomerClass', 'StageName'],
      dtype='object')


In [404]:
df_opps_final = df_opps_final[['ProjectId', 'OppId','Opportunity', 'Division', 'DealType', 'CreatedDate',
                               'DeliveryDate', 'CloseDate', 'InternalOwner', 'Estimator', 'Trades',
                               'EndMarket', 'ProjectStage', 'StageName','IsWon', 'Amount',
                               'CustomerClass', 'ContactId']]

In [405]:
df_proj_final = df_proj_final.rename(columns = {'Id':'ProjectId', 'Name':'Project', 'Project_State__c':'ProjectState'})

In [406]:
df_acc_final = df_acc_final.rename(columns = {'Account.Industry':'AccountIndustry', 'Account.Name':'AccountName',
                                              'Account.Sales_Division__c':'SalesDivision', 'Account.Type':'AccountType',
                                              'Account.Type__c':'CompanyType', 'Id':'ContactId',
                                              'Account.BillingAddress':'BillingAddress'})

In [407]:
df_acc_final = df_acc_final[['AccountId', 'AccountName', 'BillingAddress','AccountType',
                             'AccountIndustry', 'CompanyType', 'SalesDivision', 
                             'Name', 'Title', 'ContactId']]

In [408]:
df_opps_proj = pd.merge(df_opps_final, df_proj_final, on='ProjectId', how='left')

In [409]:
df_opps_proj.fillna({'Project': df_opps_proj['Opportunity']}, inplace=True)

In [410]:
df = pd.merge(df_opps_proj, df_acc_final, on='ContactId', how='left')

In [411]:
print(df.columns)

Index(['ProjectId', 'OppId', 'Opportunity', 'Division', 'DealType',
       'CreatedDate', 'DeliveryDate', 'CloseDate', 'InternalOwner',
       'Estimator', 'Trades', 'EndMarket', 'ProjectStage', 'StageName',
       'IsWon', 'Amount', 'CustomerClass', 'ContactId', 'Project',
       'ProjectState', 'AccountId', 'AccountName', 'BillingAddress',
       'AccountType', 'AccountIndustry', 'CompanyType', 'SalesDivision',
       'Name', 'Title'],
      dtype='object')


In [412]:
df = df[['ProjectId', 'Project', 'OppId', 'Opportunity', 'ProjectState', 'Division', 'DealType',
         'CreatedDate', 'DeliveryDate', 'CloseDate', 'InternalOwner', 'Estimator',
         'Trades', 'EndMarket', 'ProjectStage', 'StageName','IsWon', 'Amount', 'ContactId',
         'AccountId', 'AccountName', 'BillingAddress','AccountType', 'CustomerClass',
         'AccountIndustry', 'CompanyType', 'SalesDivision',
         'Name', 'Title']]

In [413]:
#df.to_csv(r'C:\Users\AdrielPiacquadioENG\Desktop\Estimation\Predicciones\Forecasting Opps\test.csv', index=False)

In [414]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
import pytz

In [415]:
regex_opp_1 = r'CO(-|([0-9]{1,2}))'

mask_1 = df['DealType'].isna() | (df['DealType'] == '')
mask_regex_1 = df['Opportunity'].str.contains(regex_opp_1, na=False)

df.loc[mask_1 & mask_regex_1, 'DealType'] = 'Closed Budget – Change Order'

C:\Users\AdrielPiacquadioENG\AppData\Local\Temp\ipykernel_41608\792273233.py:4: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask_regex_1 = df['Opportunity'].str.contains(regex_opp_1, na=False)


In [416]:
regex_opp_2 = r'T&M'

mask_regex_2 = df['Opportunity'].str.contains(regex_opp_2, na=False)

df.loc[mask_1 & mask_regex_2, 'DealType'] = 'T&M New Contract (with NTE)'

In [417]:
df['DealType'] = df['DealType'].fillna('Closed Budget – New Contract')

In [418]:
df = df.dropna(subset=['ContactId'])

In [419]:
df['ProjectState'] = np.where(df['ProjectState'].isnull(), df['BillingAddress'], df['ProjectState'])
df['ProjectState'] = df['ProjectState'].fillna('N/A')

In [420]:
columns = ['InternalOwner', 'SalesDivision', 'Estimator', 'Trades', 'ProjectStage', 'IsWon', 'ContactId', 'Name', 'AccountId', 'AccountName', 'AccountType', 'CustomerClass', 'ProjectState']

# Verificar si alguna de las columnas tiene valores nulos
columnas_con_nulos = df[columns].isnull().any()

# Filtrar el DataFrame para eliminar las filas con valores nulos en esas columnas
if not columnas_con_nulos.empty:
    df = df.dropna(subset=columnas_con_nulos.index)

In [421]:
print(df.columns)

Index(['ProjectId', 'Project', 'OppId', 'Opportunity', 'ProjectState',
       'Division', 'DealType', 'CreatedDate', 'DeliveryDate', 'CloseDate',
       'InternalOwner', 'Estimator', 'Trades', 'EndMarket', 'ProjectStage',
       'StageName', 'IsWon', 'Amount', 'ContactId', 'AccountId', 'AccountName',
       'BillingAddress', 'AccountType', 'CustomerClass', 'AccountIndustry',
       'CompanyType', 'SalesDivision', 'Name', 'Title'],
      dtype='object')


In [422]:
# Obtener una lista de nombres únicos en la columna 'Division'
unique_divisions = df['Division'].unique()

# Crear un diccionario que mapee cada nombre único a un identificador único
division_to_id = {division: i+1 for i, division in enumerate(unique_divisions)}

# Crear la nueva columna 'DivisionId' utilizando el diccionario de mapeo
df['DivisionId'] = df['Division'].map(division_to_id)

In [423]:
# Obtener una lista de nombres únicos en la columna 'DealType'
unique_deals = df['DealType'].unique()

# Crear un diccionario que mapee cada nombre único a un identificador único
deal_to_id = {deal: i+1 for i, deal in enumerate(unique_deals)}

# Crear la nueva columna 'InternalOwnerId' utilizando el diccionario de mapeo
df['DealTypeId'] = df['DealType'].map(deal_to_id)

In [424]:
# Obtener una lista de nombres únicos en la columna 'InternalOwner'
unique_owners = df['InternalOwner'].unique()

# Crear un diccionario que mapee cada nombre único a un identificador único
owner_to_id = {owner: i+1 for i, owner in enumerate(unique_owners)}

# Crear la nueva columna 'InternalOwnerId' utilizando el diccionario de mapeo
df['InternalOwnerId'] = df['InternalOwner'].map(owner_to_id)

In [425]:
df['SalesDivisionId'] = np.where(df['SalesDivision'] == 'Inbound', 1, 
                                 np.where(df['SalesDivision'] == 'Outbound', 2, 3))

In [426]:
# Obtener una lista de nombres únicos en la columna 'Estimator'
unique_estimators = df['Estimator'].unique()

# Crear un diccionario que mapee cada nombre único a un identificador único
estimator_to_id = {estimator: i+1 for i, estimator in enumerate(unique_estimators)}

# Crear la nueva columna 'InternalOwnerId' utilizando el diccionario de mapeo
df['EstimatorId'] = df['Estimator'].map(estimator_to_id)

In [427]:
# Obtener una lista de nombres únicos en la columna 'Trades'
unique_trades = df['Trades'].unique()

# Crear un diccionario que mapee cada nombre único a un identificador único
trade_to_id = {trade: i+1 for i, trade in enumerate(unique_trades)}

# Crear la nueva columna 'TradeId' utilizando el diccionario de mapeo
df['TradeId'] = df['Trades'].map(trade_to_id)

In [428]:
# Obtener una lista de nombres únicos en la columna 'EndMarket'
unique_endmarkets = df['EndMarket'].unique()

# Crear un diccionario que mapee cada nombre único a un identificador único
endmarket_to_id = {endmarket: i+1 for i, endmarket in enumerate(unique_endmarkets)}

# Crear la nueva columna 'EndMarketId' utilizando el diccionario de mapeo
df['EndMarketId'] = df['EndMarket'].map(endmarket_to_id)

In [429]:
df['ProjectStageId'] = np.where(df['ProjectStage'] == 'Bidding', 1, 
                                 np.where(df['ProjectStage'] == 'Awarded', 2, 3))

In [430]:
# Obtener una lista de nombres únicos en la columna 'ContactId'
unique_contacts = df['ContactId'].unique()

# Crear un diccionario que mapee cada nombre único a un identificador único
contact_to_id = {contact: i+1 for i, contact in enumerate(unique_contacts)}

# Crear la nueva columna 'ContactId_int' utilizando el diccionario de mapeo
df['ContactId_int'] = df['ContactId'].map(contact_to_id)

In [431]:
# Obtener una lista de nombres únicos en la columna 'AccountId'
unique_accounts = df['AccountId'].unique()

# Crear un diccionario que mapee cada nombre único a un identificador único
account_to_id = {account: i+1 for i, account in enumerate(unique_accounts)}

# Crear la nueva columna 'AccountId_int' utilizando el diccionario de mapeo
df['AccountId_int'] = df['AccountId'].map(account_to_id)

In [432]:
# Obtener una lista de nombres únicos en la columna 'AccountType'
unique_acctypes = df['AccountType'].unique()

# Crear un diccionario que mapee cada nombre único a un identificador único
acctype_to_id = {acctype: i+1 for i, acctype in enumerate(unique_acctypes)}

# Crear la nueva columna 'AccountTypeId' utilizando el diccionario de mapeo
df['AccountTypeId'] = df['AccountType'].map(acctype_to_id)

In [433]:
# Obtener una lista de nombres únicos en la columna 'CustomerClass'
unique_custclass = df['CustomerClass'].unique()

# Crear un diccionario que mapee cada nombre único a un identificador único
custclass_to_id = {custclass: i+1 for i, custclass in enumerate(unique_custclass)}

# Crear la nueva columna 'CustomerClassId' utilizando el diccionario de mapeo
df['CustomerClassId'] = df['CustomerClass'].map(custclass_to_id)

In [434]:
# Obtener una lista de nombres únicos en la columna 'AccountIndustry'
unique_accindustry = df['AccountIndustry'].unique()

# Crear un diccionario que mapee cada nombre único a un identificador único
accindustry_to_id = {accindustry: i+1 for i, accindustry in enumerate(unique_accindustry)}

# Crear la nueva columna 'AccountIndustryId' utilizando el diccionario de mapeo
df['AccountIndustryId'] = df['AccountIndustry'].map(accindustry_to_id)

In [435]:
# Obtener una lista de nombres únicos en la columna 'CompanyType'
unique_companytype = df['CompanyType'].unique()

# Crear un diccionario que mapee cada nombre único a un identificador único
companytype_to_id = {companytype: i+1 for i, companytype in enumerate(unique_companytype)}

# Crear la nueva columna 'CompanyTypeId' utilizando el diccionario de mapeo
df['CompanyTypeId'] = df['CompanyType'].map(companytype_to_id)

In [436]:
# Obtener una lista de nombres únicos en la columna 'ProjectState'
unique_projstate = df['ProjectState'].unique()

# Crear un diccionario que mapee cada nombre único a un identificador único
projstate_to_id = {projstate: i+1 for i, projstate in enumerate(unique_projstate)}

# Crear la nueva columna 'CustomerClassId' utilizando el diccionario de mapeo
df['ProjectStateId'] = df['ProjectState'].map(projstate_to_id)

In [437]:
df['IsWonId'] = 0  # Inicialmente, establecemos todos los valores en 0
df.loc[df['IsWon'] == True, 'IsWonId'] = 1  # Si IsWon es True, asignamos 1 a IsWonId

In [438]:
df = df[['ProjectId', 'Project', 'OppId','Opportunity', 'ProjectState', 'ProjectStateId',
         'Division', 'DivisionId', 'DealType', 'DealTypeId','CreatedDate', 'DeliveryDate', 'CloseDate',
         'InternalOwner', 'InternalOwnerId', 'SalesDivision', 'SalesDivisionId',
         'Estimator', 'EstimatorId', 'Trades', 'TradeId', 'EndMarket', 'EndMarketId',
         'ProjectStage', 'ProjectStageId', 'StageName', 'IsWon', 'IsWonId',
         'Amount', 'AccountId', 'AccountName', 'AccountId_int', 'AccountType',
         'AccountTypeId', 'CustomerClass', 'CustomerClassId', 'AccountIndustry', 'AccountIndustryId',
         'CompanyType', 'CompanyTypeId', 'ContactId', 'Name', 'ContactId_int', 'Title']]

In [439]:
df1 = df.copy()

timezone = pytz.timezone('America/Los_Angeles')
df1['CreatedDate'] = pd.to_datetime(df1['CreatedDate'])
date_df_predictions = (datetime.now() - relativedelta(months=6)).astimezone(timezone)
df1 = df1.loc[(df1['CreatedDate'] < date_df_predictions) | (df1['IsWon'] == True)]

df1.drop(columns=['ProjectId', 'OppId', 'Division', 'DeliveryDate', 'CloseDate',
                  'EndMarket', 'StageName', 'AccountIndustry', 'CompanyType', 'Title'], inplace=True)

In [440]:
print(df1.columns)

Index(['Project', 'Opportunity', 'ProjectState', 'ProjectStateId',
       'DivisionId', 'DealType', 'DealTypeId', 'CreatedDate', 'InternalOwner',
       'InternalOwnerId', 'SalesDivision', 'SalesDivisionId', 'Estimator',
       'EstimatorId', 'Trades', 'TradeId', 'EndMarketId', 'ProjectStage',
       'ProjectStageId', 'IsWon', 'IsWonId', 'Amount', 'AccountId',
       'AccountName', 'AccountId_int', 'AccountType', 'AccountTypeId',
       'CustomerClass', 'CustomerClassId', 'AccountIndustryId',
       'CompanyTypeId', 'ContactId', 'Name', 'ContactId_int'],
      dtype='object')


In [441]:
# Seleccionar las características y la variable objetivo
X = df1[['ProjectStateId', 'DealTypeId', 'InternalOwnerId',
         'SalesDivisionId', 'EstimatorId', 'TradeId',
         'ProjectStageId', 'ContactId_int', 'AccountId_int',
         'AccountTypeId', 'CustomerClassId', 'EndMarketId',
         'CompanyTypeId', 'DivisionId', 'AccountIndustryId']]
y = df1['IsWonId']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y ajustar el modelo RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Realizar predicciones sobre el conjunto de prueba
y_pred = rf_model.predict(X_test)

# Calcular métricas de clasificación
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

# Print metrics
print(f'Accuracy of the model: {accuracy * 100:.2f}%')
print(f'Precision: {precision:.2f}')
print(f'Sensibility (Recall): {recall:.2f}')
print(f'Area under ROC curve (AUC-ROC): {roc_auc:.2f}')

Accuracy of the model: 90.27%
Precision: 0.88
Sensibility (Recall): 0.88
Area under ROC curve (AUC-ROC): 0.90


In [442]:
# Obtener los valores únicos de las columnas 'InternalOwner', 'SalesDivision', etc.
unique_states = df['ProjectState'].unique()
unique_deals = df['DealType'].unique()
unique_internal_owners = df['InternalOwner'].unique()
unique_sales_divisions = df['SalesDivision'].unique()
unique_estimators = df['Estimator'].unique()
unique_trades = df['Trades'].unique()
unique_projstages = df['ProjectStage'].unique()
unique_contacts = df['Name'].unique()
unique_accounts = df['AccountName'].unique()
unique_acctypes = df['AccountType'].unique()
unique_custclasses = df['CustomerClass'].unique()
unique_endmarkets = df['EndMarket'].unique()
unique_comptypes = df['CompanyType'].unique()
unique_divisions = df['Division'].unique()
unique_accindustries = df['AccountIndustry'].unique()

# Crea un diccionario para mapear valores a IDs
state_id_map = {state: state_id for state_id, state in enumerate(unique_states, start=1)}
deal_id_map = {deal: deal_id for deal_id, deal in enumerate(unique_deals, start=1)}
internal_owner_id_map = {owner: owner_id for owner_id, owner in enumerate(unique_internal_owners, start=1)}
sales_division_id_map = {division: division_id for division_id, division in enumerate(unique_sales_divisions, start=1)}
estimator_id_map = {estimator: estimator_id for estimator_id, estimator in enumerate(unique_estimators, start=1)}
trade_id_map = {trade: trade_id for trade_id, trade in enumerate(unique_trades, start=1)}
projstage_id_map = {projstage: projstage_id for projstage_id, projstage in enumerate(unique_projstages, start=1)}
contact_id_map = {contact: contact_id for contact_id, contact in enumerate(unique_contacts, start=1)}
account_id_map = {account: account_id for account_id, account in enumerate(unique_accounts, start=1)}
acctype_id_map = {acctype: acctype_id for acctype_id, acctype in enumerate(unique_acctypes, start=1)}
custclass_id_map = {custclass: custclass_id for custclass_id, custclass in enumerate(unique_custclasses, start=1)}
endmarket_id_map = {endmarket: endmarket_id for endmarket_id, endmarket in enumerate(unique_endmarkets, start=1)}
comptype_id_map = {comptype: comptype_id for comptype_id, comptype in enumerate(unique_comptypes, start=1)}
division_id_map = {division: division_id for division_id, division in enumerate(unique_divisions, start=1)}
accindustry_id_map = {accindustry: accindustry_id for accindustry_id, accindustry in enumerate(unique_accindustries, start=1)}

# Función para obtener el ID mapeado
def get_mapped_id(value, id_map):
    return id_map.get(value)

def predecir_iswon(project_state, deal_type, internal_owner,
                   sales_division, estimator, trade,
                   project_stage, contact_id, account_id,
                   account_type, customer_class, end_market,
                   company_type, division, account_industry):
    # Mapear valores a IDs
    state_id = get_mapped_id(project_state, state_id_map)
    deal_id = get_mapped_id(deal_type, deal_id_map)
    internal_owner_id = get_mapped_id(internal_owner, internal_owner_id_map)
    sales_division_id = get_mapped_id(sales_division, sales_division_id_map)
    estimator_id = get_mapped_id(estimator, estimator_id_map)
    trade_id = get_mapped_id(trade, trade_id_map)
    projstage_id = get_mapped_id(project_stage, projstage_id_map)
    contact_id = get_mapped_id(contact_id, contact_id_map)
    account_id = get_mapped_id(account_id, account_id_map)
    acctype_id = get_mapped_id(account_type, acctype_id_map)
    custclass_id = get_mapped_id(customer_class, custclass_id_map)
    endmarket_id = get_mapped_id(end_market, endmarket_id_map)
    comptype_id = get_mapped_id(company_type, comptype_id_map)
    division_id = get_mapped_id(division, division_id_map)
    accindustry_id = get_mapped_id(account_industry, accindustry_id_map)
    # Haz lo mismo para las otras columnas

    # Crear un DataFrame con los datos de entrada
    data = pd.DataFrame({
        'ProjectStateId': [state_id],
        'DealTypeId': [deal_id],
        'InternalOwnerId': [internal_owner_id],
        'SalesDivisionId': [sales_division_id],
        'EstimatorId': [estimator_id],
        'TradeId': [trade_id],
        'ProjectStageId': [projstage_id],
        'ContactId_int': [contact_id],
        'AccountId_int': [account_id],
        'AccountTypeId': [acctype_id],
        'CustomerClassId': [custclass_id],
        'EndMarketId': [endmarket_id],
        'CompanyTypeId': [comptype_id],
        'DivisionId': [division_id],
        'AccountIndustryId': [accindustry_id]
    })

    # Realizar la predicción con el modelo
    prediction = rf_model.predict(data)
    
    # Realizar la predicción de probabilidades con el modelo
    predicted_proba = rf_model.predict_proba(data)
    
    # Obtener la probabilidad de que la predicción sea igual a 1
    prob_iswon = predicted_proba[:, rf_model.classes_ == 1]
    
    # Devolver la predicción y la probabilidad resultante
    return prediction[0], prob_iswon[0][0] * 100 # Predicción y probabilidad en porcentaje

In [443]:
df2 = df.copy()
df2 = df2.loc[(df['CreatedDate'] >= '2024-01-01')]

# Obtener las predicciones y probabilidades de conversión para cada proyecto en df2
df2['Chances'], df2['Probabilidad_IsWon'] = zip(*df2.apply(lambda row: predecir_iswon(row['ProjectState'], row['DealType'],
                                                                                      row['InternalOwner'], row['SalesDivision'],
                                                                                      row['Estimator'], row['Trades'],
                                                                                      row['ProjectStage'], row['Name'],
                                                                                      row['AccountName'], row['AccountType'],
                                                                                      row['CustomerClass'], row['EndMarket'],
                                                                                      row['CompanyType'], row['Division'],
                                                                                      row['AccountIndustry']), axis=1))

# Calcular los ingresos esperados para cada proyecto
df2['ExpectedIncome'] = df2['Probabilidad_IsWon'] * df2['Amount'] / 100

In [444]:
#df2.to_csv(r'C:\Users\AdrielPiacquadioENG\Desktop\Estimation\Predicciones\Forecasting Opps\test3.csv', index=False)

In [446]:
for index, row in df2.iterrows():
    opportunity_id = row['OppId']
    probability_value = row['Probabilidad_IsWon']
    
    # Obtener los detalles actuales de la oportunidad
    opportunity_details = sf.Opportunity.get(opportunity_id)
    
    # Verificar el valor del campo 'Are_you_using_a_package_strategy__c'
    package_strategy = opportunity_details.get('Are_you_using_a_package_strategy__c', 'No')
    
    # Si el campo está vacío, asignar 'No'
    if not package_strategy:
        package_strategy = 'No'
    
    # Actualizar la oportunidad con los nuevos valores
    sf.Opportunity.update(opportunity_id, {
        'Conversion_Chances__c': probability_value,
        'Are_you_using_a_package_strategy__c': package_strategy
    })

SalesforceMalformedRequest: Malformed request https://engbim.my.salesforce.com/services/data/v57.0/sobjects/Opportunity/006Uy0000034V8NIAU. Response content: [{'message': 'Since you marked this deal as Closed Budget – Change Order, please filll out what is the Opportunity being extended in the &quot;Extension from Opportunity&quot;field', 'errorCode': 'FIELD_CUSTOM_VALIDATION_EXCEPTION', 'fields': []}]

In [61]:
#date_test = '2024-06-01T00:00:00.000+0000'

#query_test = f"SELECT Id, Name, CreatedDate, Internal_Owner__c, IsWon, Conversion_Chances__c FROM Opportunity WHERE Project_stage__c != 'Operations' AND  CreatedDate >= {date_test}"
#response_test = sf.query(query_test)

#records_test = response_test['records']
#df_test = pd.DataFrame(records_test)

#df_test.drop(columns=['attributes'], inplace=True)

In [62]:
df_test.head(25)

,Id,Name,CreatedDate,Internal_Owner__c,IsWon,Conversion_Chances__c
0,006Uy000006zehqIAA,The Raleigh - LSS & S2B,2024-04-01T12:53:40.000+0000,Agustina Perez,False,0.000000
1,006Uy000006zizyIAA,C0650-ACC-24-Dusty Robotics CO16,2024-04-01T13:40:23.000+0000,Carmen Estrada,True,100.000000
2,006Uy000006zlD4IAI,C0650-ACC-22-Pursuit-CO106,2024-04-01T13:41:55.000+0000,Carmen Estrada,True,95.428571
3,006Uy000006zqmEIAQ,C1002-F&SM-23-ATL D16 Delta Sky Club CO03,2024-04-01T13:54:11.000+0000,Carmen Estrada,True,93.000000
4,006Uy0000070qVdIAI,C0982-CCS-24-MCPS Elementary School W Ocala,2024-04-01T17:23:45.000+0000,Agustina Perez,True,42.651515
5,006Uy0000072ClwIAE,WSU NIAR Advanced Manufacturing,2024-04-01T23:20:53.000+0000,Nicolas Videla,False,1.000000
6,006Uy0000072K8NIAU,C1030-BAT-24-Abilene Heritage Square-Spark-CO01,2024-04-01T23:59:37.000+0000,Rodrigo Pisciolari,True,1.000000
7,006Uy0000072OF8IAM,305 S. Sepulveda,2024-04-02T00:57:06.000+0000,Luciana De Simone,False,13.000000
8,006Uy0000073jW2IAI,Green Bldg - GC + COBie,2024-04-02T15:32:35.000+0000,Nicolas Prieto,False,0.000000
9,006Uy0000073t8nIAA,C1061-CJR-24-EPPLEY AIRFIELD TMP,2024-04-02T15:58:02.000+0000,Sofía Gonzalez,True,63.000000


In [136]:
#df_test.to_csv(r'C:\Users\AdrielPiacquadioENG\Desktop\Estimation\Predicciones\Forecasting Opps\test4.csv', index=False)

In [ ]:
#opportunity_id = '006Uy0000095iCvIAI'
#probability_value = 79.63

#sf.Opportunity.update(opportunity_id, {'Conversion_Chances__c': probability_value})